# Data cleaning

In [ ]:
# Imports go here

## Auxiliary functions

In [1]:
# Auxiliary code to help in the data cleaning process goes here

## 1. Cleaning the FHV dataset

### Analysis of valid values

|Column Value|Description|Data Type|Constraints|
|---	|---	|---	|---	|
|Dispatching_base_num|License Number of the base that dispatched the trip|String|Not Null|
|Pickup_datetime|The date and time of the trip pick-up|Datetime|Not Null|
|DropOff_datetime|The date and time of the trip dropoff|Datetime|Not Null|
|PULocationID|Zone in which the trip began|Integer(smallint)|Not Null|
|DOLocationID|Zone in which the trip ended|Integer(smallint)|Not Null|
|SR_Flag|Indicates if the trip was a part of a shared ride chain offered by a High Volume FHV company (e.g. Uber Pool, Lyft Line); share=1, nonshared=0|Boolean|Not Null| 

### Validity rules

### Identifying dirty records, data repairing

## 2. Cleaning the FHVHV dataset

### Analysis of valid values

|Column Value|Description|Data Type|Constraints|
|---	|---	|---	|---	|
|Hvfhs_license_num|TLC license number of the HVFHS base or business|String|Not Null|
|Dispatching_base_num|License Number of the base that dispatched the trip|String|Not Null|
|Pickup_datetime|The date and time of the trip pick-up|Datetime|Not Null|
|DropOff_datetime|The date and time of the trip dropoff|Datetime|Not Null|
|PULocationID|Zone in which the trip began|Integer(smallint)|Not Null|
|DOLocationID|Zone in which the trip ended|Integer(smallint)|Not Null|
|SR_Flag|Indicates if the trip was a part of a shared ride chain offered by a High Volume FHV company (e.g. Uber Pool, Lyft Line); share=1, nonshared=0|Boolean|Not Null| 


*Hvfhs_license_num possible values:
• HV0002: Juno
• HV0003: Uber
• HV0004: Via
• HV0005: Lyft

### Validity rules

### Identifying dirty records, data repairing

## 3. Cleaning the GREEN dataset

### Analysis of valid values

|Column Value|Description|Data Type|Constraints|
|---	|---	|---	|---	|
|VendorID|A code indicating the LPEP provider that provided the record.|Integer(tinyint)|1 or 2, Not Null|
|lpep_pickup_datetime|The date and time when the meter was engaged|Datetime|Not Null|
|lpep_dropoff_datetime|The date and time when the meter was disengaged|Datetime|Not Null|
|Passenger_count|The number of passengers in the vehicle|Integer(tinyint)|???|
|Trip_distance|The elapsed trip distance in miles reported by the taximeter|Decimal|   	|
|PULocationID|Zone in which the taximeter was engaged|Integer(smallint)|Not Null|
|DOLocationID|Zone in which the taximeter was disengaged|Integer(smallint)|Not Null|
|RateCodeID|The final rate code in effect at the end of the trip|Integer(tinyint)|1 to 6, Not Null|
|Store_and_fwd_flag|This flag indicates whether the trip record was held in vehicle memory before sending to the vendor, aka “store and forward,”because the vehicle did not have a connection to the server|Boolean|Y or N, Not Null|
|Payment_type|A numeric code signifying how the passenger paid for the trip|Integer(tinyint)|1 to 6, Not Null|
|Fare_amount|The time-and-distance fare calculated by the meter|Decimal|Not Null|
|Extra|Miscellaneous extras and surcharges|Decimal|   	|
|MTA_tax|0.50 MTA tax that is automatically triggered based on the metered rate in use|Decimal|   	|
|Improvement_surcharge|0.30 improvement surcharge assessed on hailed trips at the flag drop|Decimal|   	|
|Tip_amount|Tip amount – This field is automatically populated for credit card tips. Cash tips are not included|Decimal|   	|
|Tolls_amount|Total amount of all tolls paid in trip|Decimal|   	|
|Total_amount|The total amount charged to passengers|Decimal|   	|
|Trip_type|A code indicating whether the trip was a street-hail or a dispatch that is automatically assigned based on the metered rate in use but can be altered by the driver|Boolean|1 or 2, Not Null|

### Validity rules

### Identifying dirty records, data repairing

## 4. Cleaning the YELLOW dataset

### Analysis of valid values

### Validity rules

### Identifying dirty records, data repairing